In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import scipy.stats as st
import glob
from datetime import datetime
import time
import numba
import scipy as sc
from scipy import interpolate
import xarray as xr

import bokeh
from bokeh.models import DatetimeTickFormatter, NumeralTickFormatter, ColumnDataSource, ColorBar
from bokeh.plotting import figure
from bokeh.layouts import gridplot, row, column
from bokeh.transform import linear_cmap
from bokeh.palettes import Viridis
import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()
import holoviews as hv
import bebi103
hv.extension('bokeh')

In [ ]:
tccon_2017 = pd.read_csv('/Users/arianatribby/git/oklahoma_propane/data/tccon_retrievals/oc_alkane.vav.ada.aia.oof.csv',skiprows=290)
tccon_2018 = pd.read_csv('/Users/arianatribby/git/oklahoma_propane/data/tccon_retrievals/oc_alkane_2018.vav.ada.aia.oof.csv',skiprows=290)
tccon_2019 = pd.read_csv('/Users/arianatribby/git/oklahoma_propane/data/tccon_retrievals/oc_alkane_2019.vav.ada.aia.oof.csv',skiprows=290)
tccon_2020 = pd.read_csv('/Users/arianatribby/git/oklahoma_propane/data/tccon_retrievals/oc_alkane_2020.vav.ada.aia.oof.csv',skiprows=290)

In [ ]:
tccon_2018.head()

In [ ]:
tccon_2020.columns

In [ ]:
tccon_allyears = pd.concat([tccon_2017,tccon_2018,tccon_2019,tccon_2020])

In [ ]:
tccon_2020['pout(hPa)'].unique()

In [ ]:
tccon_allyears.year.values[-1]

In [ ]:
tccon_allyears.rename(columns = {' flag':'flag'}, inplace = True)

There is an issue where thr number of hours are greater than 24. Harrison Parker provided data without the time wrapping, and also a bool for vsf error less than arbitrary 10 since filtering those out reduced the zenith angle dependance of propane. 

In [ ]:
extra_data = pd.read_csv('/Users/arianatribby/git/oklahoma_propane/data/tccon_retrievals/VSF_error_filtering.csv')

In [ ]:
extra_data.tail()

In [ ]:
print(len(extra_data.loc[extra_data['year'] == 2017]))
print(len(tccon_2017))

In [ ]:
tccon_allyears['year'] = extra_data.year.values
tccon_allyears['month'] = extra_data.month.values
tccon_allyears['day'] = extra_data.day.values
tccon_allyears['hour'] = extra_data.hour.values
tccon_allyears['minute'] = extra_data.minute.values
tccon_allyears['second'] = extra_data.second.values
tccon_allyears['VSF_error'] = extra_data.VSF_error.values
tccon_allyears['VSF_error_lt_ten_bool'] = extra_data.VSF_error_lt_ten_bool.values

In [ ]:
tccon_allyears.year.values[-1]

#### QC - best data are flag = 0
Also, vsf error less than 10 is best to use, according to Harrison, to reduce solar zenith angle dependance

In [ ]:
tccon_qc = tccon_allyears.loc[(tccon_allyears['flag'] == 0) & 
                              (tccon_allyears['VSF_error_lt_ten_bool'] == 1)].reset_index()

In [ ]:
len(tccon_qc)

In [ ]:
tccon_allyears.loc[tccon_allyears['year'] == 2017].columns

#### Create a utc time 

In [ ]:
def create_utc(year,month,day,hour,minute,second):
    

    newdatetime = datetime.strptime(str(int(year)) + "-" + 
                      str(int(month)) + "-" +
                      str(int(day)) + "-" +
                      str(int(hour)) + "-" +
                      str(int(minute)) + "-" + 
                      str(int(second))
                    , "%Y-%m-%d-%H-%M-%S")
    
    strtime = str(newdatetime)
    datetm = datetime.strptime(strtime,'%Y-%m-%d %H:%M:%S')
    tmestamp = newdatetime.timestamp()
    
    return strtime,datetm,tmestamp

In [ ]:
# newtime = []
# newdatetime = []
# for i in range(len(NOAA_df)):
#     newtime.append(datetime.datetime(int(NOAA_df['sample_year'].values[i]),int(NOAA_df['sample_month'].values[i]),
#                       int(NOAA_df['sample_day'].values[i]),int(NOAA_df['sample_hour'].values[i]),
#                       int(NOAA_df['sample_minute'].values[i]),int(NOAA_df['sample_seconds'].values[i])).timestamp())
#     newdatetime.append(datetime.datetime(int(NOAA_df['sample_year'].values[i]),int(NOAA_df['sample_month'].values[i]),
#                       int(NOAA_df['sample_day'].values[i]),int(NOAA_df['sample_hour'].values[i]),
#                       int(NOAA_df['sample_minute'].values[i]),int(NOAA_df['sample_seconds'].values[i])))

# NOAA_df['dt_time'] = newdatetime
# NOAA_df['completetime'] = newtime

In [ ]:
def create_utc_withdayofyear(year,day,hour):
    # input time one by one, not an array
    # this takes in raw tccon time, where day is day of year, not day of month
    # and the hours are wrapped hours
    
    # check if there are hours >= 24 (I saw some, and those are not double counting day)
    if hour >= 24:
        excess_hr = abs(24-hour)
        hour = excess_hr
        day += 1   
        
    # first extract the hour min sec 
    hours = int(hour)
    minutes = (hour*60) % 60
    seconds = (hour*3600) % 60 

    
    # reformat the time 
    newdatetime = datetime.strptime(str(int(year)) + "-" + 
                      str(int(day)) + "-" +
                      str(int(hours)) + "-" +
                      str(int(minutes)) + "-" + 
                      str(int(seconds))
                    , "%Y-%j-%H-%M-%S")
    
    strtime = str(newdatetime)
    datetm = datetime.strptime(strtime,'%Y-%m-%d %H:%M:%S')
    tmestamp = newdatetime.timestamp()

    return strtime,datetm,tmestamp

In [ ]:
# if the tccon time is day of the year, do the following:
# strtimel= []
# datetml = []
# tmestampl = []
# for i in range(len(tccon_qc)):
#     strtime,datetm,tmestamp = create_utc(tccon_qc.year[i],tccon_qc.day[i],tccon_qc.hour[i])
#     strtimel.append(strtime)
#     datetml.append(datetm)
#     tmestampl.append(tmestamp)


In [ ]:
strtimel= []
datetml = []
tmestampl = []
for i in range(len(tccon_qc)):

    strtime,datetm,tmestamp = create_utc(tccon_qc.year[i],tccon_qc.month[i],tccon_qc.day[i],
                                         tccon_qc.hour[i],tccon_qc.minute[i],tccon_qc.second[i])
    strtimel.append(strtime)
    datetml.append(datetm)
    tmestampl.append(tmestamp)


In [ ]:
tccon_qc['strtime'] = strtimel
tccon_qc['datetm'] = datetml
tccon_qc['tmestamp'] = tmestampl

In [ ]:
tccon_qc.strtime.values[-1]

In [ ]:
fh = 400
fw = 400
colors = bokeh.palettes.d3['Category20'][20]

p = bokeh.plotting.figure(frame_height=fh, 
                          frame_width=fw, title='',x_axis_type='datetime')

p.circle(tccon_qc.datetm.values, tccon_qc['xc2h6(ppb)'].values, 
         size=5, fill_alpha=.5, color=colors[0], legend_label='C2H6')

p.circle(tccon_qc.datetm.values, tccon_qc['xc3h8(ppb)'].values, 
         size=5, fill_alpha=.1, color=colors[1], legend_label='C3H8')

p.xaxis.axis_label = "time"
p.yaxis.axis_label = "xC ppb"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.xaxis.major_label_orientation = 45
bokeh.io.show(p)


For both ethane and propane, there is airmass dependence (the smile)
However, propane has really low points 
Those low points could be from the priors and probably don't mean anything. (Harrison said that if there isn't a great fit, it will return the prior which is a low value.)
It also might be solar zenith angle. You can cut off solar zenith angles below 50. That variable is solzen and that would cut off the ends. 
Harrison will see if he can filter out the fits in which had a poor fit (ie returned the prior) 

### Averaging kernel data
provided by Harrison parker january 2023

In [ ]:
ethane_ak = pd.read_csv('../../data/tccon_retrievals/ethane_ak_lut.csv')
propane_ak = pd.read_csv('../../data/tccon_retrievals/propane_ak_lut.csv')
sza_bin = pd.read_csv('../../data/tccon_retrievals/SZA_bins.csv')
alt_bin = pd.read_csv('../../data/tccon_retrievals/Altitude.csv')
pres_bin = pd.read_csv("../../data/tccon_retrievals/NormalizedPressure.csv")

In [ ]:
print(len(pres_bin))
print(len(alt_bin))

In [ ]:
ethane_ak.head()

In [ ]:
# create data
data = np.array(ethane_ak)

# create coords
rows = alt_bin.values.flatten()
cols = sza_bin.values.flatten()

# put data into a dataset
ak_ds = xr.Dataset(
    data_vars=dict(
        ethane_ak=(["alt", "sza"], np.array(ethane_ak)),
        propane_ak=(["alt", "sza"], np.array(propane_ak)),
    ),
    coords=dict(
        alt=(["alt"], rows),
        sza=(["sza"], cols),
    ),
    attrs=dict(description=" "),
)

In [ ]:
ak_ds

In [ ]:
ak_ds.to_netcdf('../../data/tccon_retrievals/averaging_kforstilt.nc4')

#### TCCON prep for STILT
- Get tccon on same days that you are doing the noaa for. You can obviously do additional days using tccon but you can start with the NOAA days. 
- only look at tccon times between 10am and 2pm. 
- decide aggregate for tccon data, since including all of it will be too much. Maybe every 30 min or hourly averages, need to see with what frequency you get enhancements. 
- The additional days you want to look at are days with really high and low enhancement
- You will need to rerun stilt for tccon overlap days because you run stilt in such a way as to represent the entire column 
- we will start out at a single tccon time for ease 
- Save this out as a csv file, lat, lon, time, height, for stilt 

In [ ]:
tccon_2017qc = tccon_qc.loc[tccon_qc['year'] == 2017]

In [ ]:
fh = 400
fw = 400

data = tccon_2017qc.loc[tccon_2017qc['month'] == 6]
colors = bokeh.palettes.d3['Category20'][20]

p = bokeh.plotting.figure(frame_height=fh, 
                          frame_width=fw, title='',x_axis_type='datetime')

p.circle(data.datetm.values, data['xc2h6(ppb)'].values, 
         size=5, fill_alpha=.5, color=colors[0], legend_label='C2H6 whole record')

p.xaxis.axis_label = "time"
p.yaxis.axis_label = "xC ppb"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.legend.location = "bottom_right"
bokeh.io.show(p)

Based on zooming in between 6/15 and 6/16, it seems that every hour would capture most variability that is relevant  

but i think it would probably be better if you did the hourly averages for all the data at once. Then after you do that, you can go to the noaa fresh for stilt and filter biomass, then bring that new file here and do crossover between the same days. then save out individual files for each year.  

In [ ]:
tccon_2017qc.columns

In [ ]:
tccon_hourly_med = tccon_qc.groupby(["year", "month", "day", "hour"]).median().reset_index()

In [ ]:
tccon_hourly_med.tail()

So using thr groupby did not transfer the utc strings that you made before. Going to re-run using the new time averages. 

In [ ]:
strtimel= []
datetml = []
tmestampl = []
for i in range(len(tccon_hourly_med)):

    strtime,datetm,tmestamp = create_utc(tccon_hourly_med.year[i],tccon_hourly_med.month[i],tccon_hourly_med.day[i],
                                         tccon_hourly_med.hour[i],tccon_hourly_med.minute[i],tccon_hourly_med.second[i])
    strtimel.append(strtime)
    datetml.append(datetm)
    tmestampl.append(tmestamp)


In [ ]:
tccon_hourly_med['strtime'] = strtimel
tccon_hourly_med['datetm'] = datetml
tccon_hourly_med['tmestamp'] = tmestampl

Only choose 2017 for now since the noaa file below isn't updated beyond 2018 yet 

In [ ]:
tccon_2017qc = tccon_hourly_med.loc[tccon_hourly_med['year'] == 2017]

### Get crossover between noaa and tccon times 

In [ ]:
noaa_fresh_forstilt = pd.read_csv('../../data/windrose/noaa_summer2017tower_varsforfootprint.csv')

In [ ]:
def find_similar_times(tccon,noaa):
    # takes noaa one at a time, tccon the entire array 

    newarray = abs(tccon - noaa)
    return(np.argmin(newarray))
    
    

In [ ]:
tccon_indices = [find_similar_times(tccon_2017qc.tmestamp.values,x) for x in noaa_fresh_forstilt.completetime.values]
print(tccon_indices)

But you want to conserve all available times that match the corresponding day/month to those times. For each index, find month, day corresponding. 

In [ ]:
allnewindices = []
for i in tccon_indices: 
    year = tccon_2017qc.iloc[i].year
    month = tccon_2017qc.iloc[i].month
    day = tccon_2017qc.iloc[i].day
    inds = tccon_2017qc.loc[(tccon_2017qc['year'] == year) & 
                            (tccon_2017qc['month'] == month) & 
                            (tccon_2017qc['day'] == day)].index.values
    allnewindices.append(inds)
    

In [ ]:
flat_list = [item for sublist in allnewindices for item in sublist]

There are lots of noaa fresh measurements that repeat on the same day, so you now have duplicates in allnewindices.  

In [ ]:
inds_for_alltccontimes = np.unique(flat_list)

In [ ]:
tccon_toweroverlap2017 = tccon_2017qc.iloc[inds_for_alltccontimes]

#### How do the tccon and noaa overlaps compare to the entire timeline? 

In [ ]:
fh = 400
fw = 400
colors = bokeh.palettes.d3['Category20'][20]

p = bokeh.plotting.figure(frame_height=fh, 
                          frame_width=fw, title='',x_axis_type='datetime')

p.circle(tccon_2017qc.datetm.values, tccon_2017qc['xc2h6(ppb)'].values, 
         size=5, fill_alpha=.5, color=colors[0], legend_label='C2H6 whole record')

p.circle(tccon_toweroverlap2017.datetm.values, tccon_toweroverlap2017['xc2h6(ppb)'].values, 
         size=5, fill_alpha=.5, color=colors[2], legend_label='C2H6 noaa overlaps')

p.xaxis.axis_label = "time"
p.yaxis.axis_label = "xC ppb"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.legend.location = "bottom_right"
bokeh.io.show(p)

In [ ]:
fh = 400
fw = 400
colors = bokeh.palettes.d3['Category20'][20]

p = bokeh.plotting.figure(frame_height=fh, 
                          frame_width=fw, title='',x_axis_type='datetime')

p.circle(tccon_2017qc.datetm.values, tccon_2017qc['xc3h8(ppb)'].values, 
         size=5, fill_alpha=.5, color=colors[0], legend_label='C3H8 whole record')

p.circle(tccon_toweroverlap2017.datetm.values, tccon_toweroverlap2017['xc3h8(ppb)'].values, 
         size=5, fill_alpha=.5, color=colors[2], legend_label='C3H8 noaa overlaps')

p.xaxis.axis_label = "time"
p.yaxis.axis_label = "xC ppb"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.legend.location = "bottom_right"
bokeh.io.show(p)

#### How do observations look for different hours of the day 

In [ ]:
# tccon_2017qc['day_int'] = [int(x) for x in tccon_2017qc['day']]
# tccon_overlapwholedaynoaa2017 = tccon_2017qc.loc[tccon_2017qc['day_int'].isin(list(map(int,tccon_overlapnoaa2017.day.unique())))]

In [ ]:
tccon_toweroverlap2017.columns

UTC is 6 hours later than CST (oklahoma). So if it is 9pm on Monday utc, it is 3pm monday cst.  

In [ ]:
fh = 450
fw = 450
colors = bokeh.palettes.d3['Category20'][20]

p = bokeh.plotting.figure(frame_height=fh, 
                          frame_width=fw, title='')

p.circle(tccon_toweroverlap2017.hour.values - 6, tccon_toweroverlap2017['xc3h8(ppb)'].values, 
         size=5, fill_alpha=.5, color=colors[2], legend_label='tccon C3H8 overlaps w/noaa entire day')

p.xaxis.axis_label = "local hour of day"
p.yaxis.axis_label = "xC ppb"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.legend.location = "top_right"
bokeh.io.show(p)

keep all observations for now 

Harrison: column of water calcs to exclude monsoon days 



In [ ]:
tccon_toweroverlap2017.datetm

In [ ]:
tccon_toweroverlap2017.columns

In [ ]:
towertocsv = tccon_toweroverlap2017[['lat(deg)','long(deg)','datetm','pout(hPa)']]

In [ ]:
towertocsv.to_csv('../../data/tccon_retrievals/tccon_noaa2017toweroverlap_forfootprint.csv')

In [ ]:
# towertocsv = noaatowersummer[['lat','lon','dt_time','completetime','sample_intake_height']]

In [ ]:
# towertocsv.to_csv('../../data/tccon_retrievals/tccon_noaa2017toweroverlap_forfootprint.csv')

# All years TCCON data preparation

### Get crossover between noaa and tccon times 

In [ ]:
noaa_fresh_allyears = pd.read_csv('/Users/arianatribby/git/oklahoma_propane/data/processing/newNOAA_freshemiss2016to2020.csv')


In [ ]:
tccon_indices_allyears = [find_similar_times(tccon_hourly_med.tmestamp.values,x) for x in noaa_fresh_allyears.completetime.values]

print(tccon_indices_allyears)

But you want to conserve all available times that match the corresponding day/month to those times. For each index, find month, day corresponding. 

In [ ]:
allnewindices = []
for i in tccon_indices_allyears: 
    year = tccon_hourly_med.iloc[i].year
    month = tccon_hourly_med.iloc[i].month
    day = tccon_hourly_med.iloc[i].day
    inds = tccon_hourly_med.loc[(tccon_hourly_med['year'] == year) & 
                            (tccon_hourly_med['month'] == month) & 
                            (tccon_hourly_med['day'] == day)].index.values
    allnewindices.append(inds)
    

In [ ]:
flat_list = [item for sublist in allnewindices for item in sublist]

There are lots of noaa fresh measurements that repeat on the same day, so you now have duplicates in allnewindices.  

In [ ]:
inds_for_alltccontimes = np.unique(flat_list)

In [ ]:
tccon_overlap2017to2020 = tccon_hourly_med.iloc[inds_for_alltccontimes]

#### How do the tccon and noaa overlaps compare to the entire timeline? 

In [ ]:
fh = 400
fw = 400
colors = bokeh.palettes.d3['Category20'][20]

p = bokeh.plotting.figure(frame_height=fh, 
                          frame_width=fw, title='',x_axis_type='datetime')

p.circle(tccon_hourly_med.datetm.values, tccon_hourly_med['xc2h6(ppb)'].values, 
         size=5, fill_alpha=.5, color=colors[0], legend_label='C2H6 whole record')

p.circle(tccon_overlap2017to2020.datetm.values, tccon_overlap2017to2020['xc2h6(ppb)'].values, 
         size=5, fill_alpha=.5, color=colors[2], legend_label='C2H6 noaa overlaps')

p.xaxis.axis_label = "time"
p.yaxis.axis_label = "xC ppb"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.legend.location = "top_left"
p.xaxis.major_label_orientation = 45
bokeh.io.show(p)

In [ ]:
fh = 400
fw = 400
colors = bokeh.palettes.d3['Category20'][20]

p = bokeh.plotting.figure(frame_height=fh, 
                          frame_width=fw, title='',x_axis_type='datetime')

p.circle(tccon_hourly_med.datetm.values, tccon_hourly_med['xc3h8(ppb)'].values, 
         size=5, fill_alpha=.5, color=colors[0], legend_label='C3H8 whole record')

p.circle(tccon_overlap2017to2020.datetm.values, tccon_overlap2017to2020['xc3h8(ppb)'].values, 
         size=5, fill_alpha=.5, color=colors[2], legend_label='C3H8 noaa overlaps')

p.xaxis.axis_label = "time"
p.yaxis.axis_label = "xC ppb"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.legend.location = "top_left"
p.xaxis.major_label_orientation = 45
bokeh.io.show(p)

#### How do observations look for different hours of the day 

UTC is 6 hours later than CST (oklahoma). So if it is 9pm on Monday utc, it is 3pm monday cst.  

In [ ]:
fh = 450
fw = 450
colors = bokeh.palettes.d3['Category20'][20]

p = bokeh.plotting.figure(frame_height=fh, 
                          frame_width=fw, title='')

p.circle(tccon_overlap2017to2020.hour.values - 6, tccon_overlap2017to2020['xc3h8(ppb)'].values, 
         size=5, fill_alpha=.5, color=colors[2], legend_label='tccon C3H8 overlaps w/noaa entire day')

p.xaxis.axis_label = "local hour of day"
p.yaxis.axis_label = "xC ppb"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label_text_font_size = "16pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"
p.xaxis.major_tick_line_width = 3
p.yaxis.major_tick_line_width = 3
p.axis.axis_label_text_font_style = 'bold'
p.legend.location = "top_right"
bokeh.io.show(p)

keep all observations for now 

Harrison: column of water calcs to exclude monsoon days 



In [ ]:
tccon_overlap2017to2020.datetm

In [ ]:
tccon_overlap2017to2020.columns

In [ ]:
tccontocsv = tccon_overlap2017to2020[['lat(deg)','long(deg)','datetm','pout(hPa)']]

In [ ]:
tccontocsv.to_csv('/Users/arianatribby/git/oklahoma_propane/data/windrose/tccon_summer2017to2020summer_forstilt.csv')



#### STILT stuff 
- You will need to rerun stilt for those overlap days because release particles at a specific height for tccon. And you will have more dense times for the single flask time. 
- Save this out as a csv file, lat, lon, time, height, for stilt 

Dien said for tccon that (11/22/week1 lab notebook) 
- for tccon you could use wind profile for the entire column
- tccon has wind recorded here 
- do tccon and flasks all in the same wind rose
- for the column, only do 2km (or the height of pbl) <- i think that means that is where you should release the particles? like it should be within the boundary layer? Because xgas is the sum of the entire column but the majority is near the surface. 